# **Esempio utlizzo package TextMatcherToolkit**

**Step 1**: Installation.

In [1]:
pip install textmatchertoolkit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.6 MB/s eta 0:00:00


**Step 2**: importing from the TextMatcherToolkit package the Matcher and Utility classes.

In [2]:
from textmatchertoolkit.toolkit import Matcher
from textmatchertoolkit.toolkit import Utility

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Step 3**: Let's initialize an istance for both the Matcher and Utility classes.

In [10]:
m=Matcher()
u=Utility()

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Let's import now a toy example that consists in a FED statement and a set of tweets published in the next 48 hours by 3 timelines. We wil use it to show how to utilize the TextMatcherToolkit package.

In [11]:
import json
# Load the JSON file into a dictionary
with open('similarity_sample0.json', 'r') as f:
    similarity = json.load(f)

source=" "
candidates=[]
for dir in similarity['0']:
  if dir=="statement":
    source=similarity['0'][dir]
  if dir=='related tweets':
    for x in similarity['0'][dir]:
      candidates.append(x[2])

Now let's compute the tfidf matrix for our corpus (source+tweets). This will be useful for when we will compute the cosine similarity between the tf-idf vectors of the senetences that have to be matched.

In [12]:
import pandas as pd

In [13]:
corpus=[source]+candidates
tfidf_matrix,f=u.compute_tfidf_matrix(corpus)
pd.DataFrame(tfidf_matrix.toarray(),columns=f).head()

,account,action,activity,ad,addition,administr,administration,affect,agency,ago,...,world,wound,xstrata,yahoo,ye,year,yellen,yen,york,zero
0,0.0,0.040119,0.080238,0.0,0.0,0.0,0.0,0.0,0.036921,0.0,...,0.0,0.0,0.0,0.0,0.0,0.032891,0.040119,0.0,0.0,0.0
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


Since we want to match a sentence with a sentence, we will split the FED statement in senteces. This is not done with the tweets due to their nature (they are short, usually a single sentence).

In [14]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

sentences = sent_tokenize(source)
finals_source=[s for s in sentences]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Step 4**: Now we can finally "match" ora sentence between them. The "match" will be between each tweet with each senetence of the FED statement. Since the sentences in the FED statement are 17, and the tweets are 117, we will have a total of 117*17=1989 matches. For each of them we will output:

*   A similarity score between the two
*   The distance between their topic distributions
*   A paraphrase score
*   An entailement score
*   The positional distance between the two
*   The cosine similarity between the tf-idf vector of the two
*   A subjectivity score for each of the two sentences

Let's make a small example, comparing all the 17 senteces of the FED statement with the 1st tweet of our list "candidates".




In [15]:
sent_Tw=candidates[0]

final_result=[]
for sent_S in finals_source:
  dic={}
  dic['source_index']=finals_source.index(sent_S)
  dic['candidate_index']=candidates.index(sent_Tw)
  dic['similarity']=m.compute_similarity(sent_S,sent_Tw)
  dic['topic_coherence']=m.compute_topic_coherence(sent_S,sent_Tw)
  dic['paraphrase']=m.compute_paraphrase(sent_S,sent_Tw)
  dic['entailment']=m.compute_entailment(sent_S,sent_Tw)
  dic['pos_distance']=m.compute_positional_distance(sent_S,sent_Tw)
  dic['TF-IDF']=m.compute_tfidf_similarity(sent_S,sent_Tw,u.vectorizer)
  dic['Sub_source_sentence']=m.compute_subjectivity(sent_S)
  dic['Sub_candidate_sentence']=m.compute_subjectivity(sent_Tw)
  final_result.append(dic)

final_result[0]

{'source_index': 0,
 'candidate_index': 0,
 'similarity': 0.18972741,
 'topic_coherence': 0.3651817006163154,
 'paraphrase': 0.006242738105356693,
 'entailment': 0.014161132276058197,
 'pos_distance': 15,
 'TF-IDF': 0.0,
 'Sub_source_sentence': 0.25430965423583984,
 'Sub_candidate_sentence': 0.2191915512084961}